In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import os, json, pathlib, shutil, PIL
import matplotlib.pyplot as plt
import cv2
import itertools


In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image

In [3]:
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input as vgg_preprocess_input, decode_predictions as vgg_decode_predictions
from tensorflow.keras.applications.resnet_v2 import ResNet50V2, preprocess_input as resnet_preprocess_input, decode_predictions as resnet_decode_predictions
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input as inception_preprocess_input, decode_predictions as inception_decode_predictions
from tensorflow.keras.applications.densenet import DenseNet121, preprocess_input as densenet_preprocess_input, decode_predictions as densenet_decode_predictions

In [4]:
# preparing all the pre-trained models

vgg_layer = VGG16(weights= 'imagenet', include_top= False, input_shape=(200, 200, 3))
vgg_full_layer = VGG16(weights= 'imagenet', include_top= True, input_shape= None, classifier_activation='softmax')

resnet_layer = ResNet50V2(weights= 'imagenet', include_top= False, input_shape=(200, 200, 3))
resnet_full_layer = ResNet50V2(weights= 'imagenet', include_top= True, input_shape= None, classifier_activation='softmax')

inception_layer = InceptionV3(weights= 'imagenet', include_top= False, input_shape=(200, 200, 3))
inception_full_layer = InceptionV3(weights= 'imagenet', include_top= True, input_shape= None, classifier_activation='softmax')

densenet_layer = DenseNet121(weights= 'imagenet', include_top= False, input_shape=(200, 200, 3))
densenet_full_layer = DenseNet121(weights= 'imagenet', include_top= True, input_shape= None)

102875136/102869336 [==============================] - 4s 0us/step


In [64]:
# getting into the files
folder_dir = "Places365_train"
val_file_dir = pathlib.Path(folder_dir)
print(val_file_dir.exists())

True


In [65]:
total_img_cnt = len(list(val_file_dir.glob("*/*.jpg")))
print(total_img_cnt)
'''random_discovery_cnt = len(list(pathlib.Path(val_file_dir, 'random_discovery').glob("*.jpg")))
print(random_discovery_cnt)

total_img_cnt = total_img_cnt - random_discovery_cnt
print(total_img_cnt)'''

1500


'random_discovery_cnt = len(list(pathlib.Path(val_file_dir, \'random_discovery\').glob("*.jpg")))\nprint(random_discovery_cnt)\n\ntotal_img_cnt = total_img_cnt - random_discovery_cnt\nprint(total_img_cnt)'

In [66]:
class_names = [name for name in os.listdir(folder_dir) if os.path.isdir(os.path.join(folder_dir, name))]
#class_names.remove("random_discovery")
print(class_names)

['airfield', 'amusement-park', 'aquarium', 'bathroom', 'bus-station-indoor', 'canal-urban', 'church-outdoor', 'computer-room', 'crosswalk', 'gas-station', 'golf-course', 'gymnasium-indoor', 'restaurant-kitchen', 'tree-house', 'vineyard']


In [67]:
total_nums = []
for class_name in class_names:
  class_dir = pathlib.Path(folder_dir, class_name)
  #each_class_imgs = list(class_dir.glob("*.jpg"))
  cl_length = len(list(class_dir.glob("*.jpg")))
  train_ratio = int(cl_length * 0.7)
  test_ratio = int(cl_length * 0.2)
  valid_ratio = cl_length - (train_ratio + test_ratio)

  nums = np.zeros(cl_length)
  nums[:test_ratio] = 1
  nums[test_ratio : test_ratio + valid_ratio] = 2
  np.random.shuffle(nums)
  total_nums.append(list(nums))

merged_nums = list(itertools.chain.from_iterable(total_nums))

In [68]:
vgg_final_layer = list(vgg_layer.layers)[-1].output_shape
vgg_final_layer = list(vgg_final_layer)
print("final layer of VGG16 : " +  str(list(vgg_layer.layers)[-1]) + " and its shape : " + str(vgg_final_layer))

vgg_conv_layers = []
for l in range(len(vgg_layer.layers)):
  layer = vgg_layer.layers[l]
  if 'Conv' not in layer.__class__.__name__:
    continue
  vgg_conv_layers.append((layer.name, layer.output.shape))

vgg_conv_base_shape = []
for i in vgg_final_layer:
  if i != None:
    vgg_conv_base_shape.append(i)
print("conv_base_shape : ", vgg_conv_base_shape)

vgg_feat_shape = tuple([total_img_cnt] + vgg_conv_base_shape)
print(vgg_feat_shape)

vgg_input_dimension = np.prod(vgg_conv_base_shape)
print(vgg_input_dimension)

final layer of VGG16 : <tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x000001BFE9772A48> and its shape : [None, 6, 6, 512]
conv_base_shape :  [6, 6, 512]
(1500, 6, 6, 512)
18432


In [69]:
resnet_final_layer = list(resnet_layer.layers)[-1].output_shape
resnet_final_layer = list(resnet_final_layer)

resnet_conv_layers = []
for l in range(len(resnet_layer.layers)):
  layer = resnet_layer.layers[l]
  if 'Conv' not in layer.__class__.__name__:
    continue
  resnet_conv_layers.append((layer.name, layer.output.shape))

resnet_conv_base_shape = []
for i in resnet_final_layer:
  if i != None:
    resnet_conv_base_shape.append(i)
print("conv_base_shape : ", resnet_conv_base_shape)

resnet_feat_shape = tuple([total_img_cnt] + resnet_conv_base_shape)
print(resnet_feat_shape)

resnet_input_dimension = np.prod(resnet_conv_base_shape)
print(resnet_input_dimension)

conv_base_shape :  [7, 7, 2048]
(1500, 7, 7, 2048)
100352


In [70]:
inception_final_layer = list(inception_layer.layers)[-1].output_shape
inception_final_layer = list(inception_final_layer)

inception_conv_layers = []
for l in range(len(inception_layer.layers)):
  layer = inception_layer.layers[l]
  if 'Conv' not in layer.__class__.__name__:
    continue
  inception_conv_layers.append((layer.name, layer.output.shape))

inception_conv_base_shape = []
for i in inception_final_layer:
  if i != None:
    inception_conv_base_shape.append(i)
print("conv_base_shape : ", inception_conv_base_shape)

inception_feat_shape = tuple([total_img_cnt] + inception_conv_base_shape)
print(inception_feat_shape)

inception_input_dimension = np.prod(inception_conv_base_shape)
print(inception_input_dimension)

conv_base_shape :  [4, 4, 2048]
(1500, 4, 4, 2048)
32768


In [71]:
densenet_final_layer = list(densenet_layer.layers)[-1].output_shape
densenet_final_layer = list(densenet_final_layer)

densenet_conv_layers = []
for l in range(len(densenet_layer.layers)):
  layer = densenet_layer.layers[l]
  if 'Conv' not in layer.__class__.__name__:
    continue
  densenet_conv_layers.append((layer.name, layer.output.shape))

densenet_conv_base_shape = []
for i in densenet_final_layer:
  if i != None:
    densenet_conv_base_shape.append(i)
print("conv_base_shape : ", densenet_conv_base_shape)

densenet_feat_shape = tuple([total_img_cnt] + densenet_conv_base_shape)
print(densenet_feat_shape)

densenet_input_dimension = np.prod(densenet_conv_base_shape)
print(densenet_input_dimension)

conv_base_shape :  [6, 6, 1024]
(1500, 6, 6, 1024)
36864


In [72]:
print(vgg_conv_layers[-1])
print(resnet_conv_layers[-1])
print(inception_conv_layers[-1])
print(densenet_conv_layers[-1])

('block5_conv3', TensorShape([None, 12, 12, 512]))
('conv5_block3_3_conv', TensorShape([None, 7, 7, 2048]))
('conv2d_93', TensorShape([None, 4, 4, 192]))
('conv5_block16_2_conv', TensorShape([None, 6, 6, 32]))


In [73]:
datagen = ImageDataGenerator(rescale=1./255)
batch_size = 50

In [74]:
# leave out generator, and create all 4 models' feature at one function
generator = datagen.flow_from_directory(val_file_dir, target_size=(200, 200), batch_size = batch_size, class_mode= 'categorical', shuffle=False)

filepaths = []
for filepath in generator.filepaths:
  filepaths.append(filepath)
  
filenames = []
for filename in generator.filenames:
  filenames.append(filename)

Found 1500 images belonging to 15 classes.


In [75]:
ground_truth_label = []
file_names = []
for file in filenames:
  f = file.split("\\")
  ground_truth_label.append(f[0])
  file_names.append(f[1])  

index_list = list(range(0, total_img_cnt))

predicting with whole pre-trained model

In [25]:
# predicting with whole pre-trained model 

def pretrained_prediction(img_paths, target_size, classifier, preprocess_input, decode_predictions):
  whole_images = []
  
  for img_path in img_paths:
    raw_img = image.load_img(img_path, target_size= target_size)
    img = image.img_to_array(raw_img)
    img = np.expand_dims(img, axis = 0)
    img = preprocess_input(img)
    img = whole_images.append(img)

  whole_images = np.vstack(whole_images)
  prediction = classifier.predict(whole_images, batch_size = batch_size)
  top_k = decode_predictions(prediction, top = 10)

  return prediction, top_k, whole_images

  

In [26]:
vgg_pred, vgg_topk, vgg_processed_imgs = pretrained_prediction(filepaths, (224, 224), vgg_full_layer, vgg_preprocess_input, vgg_decode_predictions)
resnet_pred, resnet_topk, resnet_processed_imgs = pretrained_prediction(filepaths, (224, 224), resnet_full_layer, resnet_preprocess_input, resnet_decode_predictions)

KeyboardInterrupt: 

In [ ]:
inception_pred, inception_topk, inception_processed_imgs = pretrained_prediction(filepaths, (299, 299), inception_full_layer, inception_preprocess_input, inception_decode_predictions)
densenet_pred, densenet_topk, densenet_processed_imgs = pretrained_prediction(filepaths, (224, 224), densenet_full_layer, densenet_preprocess_input, densenet_decode_predictions)

In [76]:
def extract_features(generator, data_num, class_num, feature_shape, pretrained_model):
  features = np.zeros(shape = feature_shape)
  labels = np.zeros(shape=(data_num, class_num))
  #generator = datagen.flow_from_directory(directory, target_size=(224, 224), batch_size = batch_size, class_mode= 'categorical', shuffle=False)
  i = 0
  for inputs_batch, labels_batch in generator:
    features_batch = pretrained_model.predict(inputs_batch)
    features[i * batch_size : (i+1) * batch_size] = features_batch
    labels[i * batch_size : (i+1) * batch_size] = labels_batch
    i += 1
    if i * batch_size >= data_num:
      break

  return features, labels

In [77]:
vgg_features, vgg_labels = extract_features(generator, total_img_cnt, len(class_names), vgg_feat_shape, vgg_layer)
resnet_features, resnet_labels = extract_features(generator, total_img_cnt, len(class_names), resnet_feat_shape, resnet_layer)
inception_features, inception_labels = extract_features(generator, total_img_cnt, len(class_names), inception_feat_shape, inception_layer)
densenet_features, densenet_labels = extract_features(generator, total_img_cnt, len(class_names), densenet_feat_shape, densenet_layer)

In [78]:
total_labels_int = []
for idx in range(len(vgg_labels)):
  total_labels_int.append(np.argmax(vgg_labels[idx]))

In [79]:
data_df = pd.DataFrame({'index': index_list, 'label': ground_truth_label, 'label_int': total_labels_int, 'file_names' : file_names, 'file_path' : filepaths})
pretrained_data_df = pd.DataFrame({'index': index_list, 'label': ground_truth_label, 'label_int': total_labels_int, 'file_names' : file_names, 'file_path' : filepaths})

In [ ]:
pretrained_data_df['vgg_pred_score'] = list(vgg_pred)
pretrained_data_df['vgg_topk_pred'] = list(vgg_topk)
pretrained_data_df['resnet_pred_score'] = list(resnet_pred)
pretrained_data_df['resnet_topk_pred'] = list(resnet_topk)
pretrained_data_df['inception_pred_score'] = list(inception_pred)
pretrained_data_df['inception_topk_pred'] = list(inception_topk)
pretrained_data_df['densenet_pred_score'] = list(densenet_pred)
pretrained_data_df['densenet_topk_pred'] = list(densenet_topk)

In [ ]:
result = pretrained_data_df.to_json(orient="records")
parsed = json.loads(result)
with open("pretrained_places365_result.json", "w") as outfile:
  json.dump(parsed, outfile, indent = 4)

In [80]:
vgg_features = np.reshape(vgg_features, (total_img_cnt, vgg_input_dimension))
resnet_features = np.reshape(resnet_features, (total_img_cnt, resnet_input_dimension))
inception_features = np.reshape(inception_features, (total_img_cnt, inception_input_dimension))
densenet_features = np.reshape(densenet_features, (total_img_cnt, densenet_input_dimension))

In [81]:
from sklearn.cluster import KMeans

CLUSTER_INT = 3

vgg_clusters = KMeans(CLUSTER_INT, random_state = 0)
vgg_clusters.fit(vgg_features)

resnet_clusters = KMeans(CLUSTER_INT, random_state= 0)
resnet_clusters.fit(resnet_features)

inception_clusters = KMeans(CLUSTER_INT, random_state = 0)
inception_clusters.fit(inception_features)

densenet_clusters = KMeans(CLUSTER_INT, random_state= 0)
densenet_clusters.fit(densenet_features)

KMeans(n_clusters=3, random_state=0)

In [82]:
models = []

In [83]:
# Add classifier on pre-trained model
vgg_model = keras.models.Sequential()
#vgg_model.add(keras.layers.Reshape((vgg_input_dimension,), input_shape = tuple(vgg_conv_base_shape)))
vgg_model.add(keras.layers.Dense(512, activation='relu', input_dim = vgg_input_dimension))
vgg_model.add(keras.layers.Dense(512, activation='relu'))
vgg_model.add(keras.layers.Dense(len(class_names), activation = 'softmax'))
vgg_model.compile(optimizer=keras.optimizers.Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

models.append(vgg_model)

In [84]:
resnet_model = keras.models.Sequential()
#resnet_model.add(keras.layers.Reshape((resnet_input_dimension,), input_shape = tuple(resnet_conv_base_shape)))
resnet_model.add(keras.layers.Dense(512, activation='relu', input_dim = resnet_input_dimension))
resnet_model.add(keras.layers.Dense(512, activation='relu'))
resnet_model.add(keras.layers.Dense(len(class_names), activation = 'softmax'))
resnet_model.compile(optimizer=keras.optimizers.Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

models.append(resnet_model)

In [85]:
inception_model = keras.models.Sequential()
#inception_model.add(keras.layers.Reshape((inception_input_dimension,), input_shape = tuple(inception_conv_base_shape)))
inception_model.add(keras.layers.Dense(512, activation='relu', input_dim = inception_input_dimension))
inception_model.add(keras.layers.Dense(512, activation='relu'))
inception_model.add(keras.layers.Dense(len(class_names), activation = 'softmax'))
inception_model.compile(optimizer=keras.optimizers.Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

models.append(inception_model)

In [86]:
densenet_model = keras.models.Sequential()
#densenet_model.add(keras.layers.Reshape((densenet_input_dimension,), input_shape = tuple(densenet_conv_base_shape)))
densenet_model.add(keras.layers.Dense(512, activation='relu', input_dim = densenet_input_dimension))
densenet_model.add(keras.layers.Dense(512, activation='relu'))
densenet_model.add(keras.layers.Dense(len(class_names), activation = 'softmax'))
densenet_model.compile(optimizer=keras.optimizers.Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

models.append(densenet_model)

In [87]:
'''
nums = np.zeros(total_img_cnt)
train_ratio = int(total_img_cnt * 0.6)
test_ratio = int(total_img_cnt * 0.2)
valid_ratio = total_img_cnt - (train_ratio + test_ratio)

print(train_ratio)
print(test_ratio)
print(valid_ratio)

nums[:test_ratio] = 1
nums[test_ratio:test_ratio + valid_ratio] = 2
np.random.shuffle(nums)'''

pmt_order = np.random.permutation(np.arange(total_img_cnt))

In [88]:
#put clustering result

In [89]:
vgg_data_df = pd.DataFrame({'index': index_list, 'file_names': file_names, 'feature' : list(vgg_features), 'label' : list(vgg_labels), 'int_label' : total_labels_int, 'cluster': vgg_clusters.labels_, 'assign' : merged_nums})
resnet_data_df = pd.DataFrame({'index': index_list, 'file_names': file_names, 'feature' : list(resnet_features), 'label' : list(resnet_labels), 'int_label' : total_labels_int, 'cluster': resnet_clusters.labels_, 'assign' : merged_nums})
inception_data_df = pd.DataFrame({'index': index_list, 'file_names': file_names, 'feature' : list(inception_features), 'label' : list(inception_labels), 'int_label' : total_labels_int, 'cluster': inception_clusters.labels_, 'assign' : merged_nums})
densenet_data_df = pd.DataFrame({'index': index_list, 'file_names': file_names, 'feature' : list(densenet_features), 'label' : list(densenet_labels), 'int_label' : total_labels_int,'cluster': densenet_clusters.labels_, 'assign' : merged_nums})

In [90]:
data_df['assign'] = merged_nums
data_df['vgg_clusters'] = vgg_clusters.labels_
data_df['resnet_clusters'] = resnet_clusters.labels_
data_df['inception_clusters'] = inception_clusters.labels_
data_df['densenet_clusters'] = densenet_clusters.labels_

In [91]:
data_df.head()

,index,label,label_int,file_names,file_path,assign,vgg_clusters,resnet_clusters,inception_clusters,densenet_clusters
0,0,airfield,0,Places365_val_00000435.jpg,Places365_train\airfield\Places365_val_0000043...,1.0,0,0,2,1
1,1,airfield,0,Places365_val_00001163.jpg,Places365_train\airfield\Places365_val_0000116...,0.0,1,0,2,1
2,2,airfield,0,Places365_val_00001257.jpg,Places365_train\airfield\Places365_val_0000125...,0.0,0,0,2,1
3,3,airfield,0,Places365_val_00001316.jpg,Places365_train\airfield\Places365_val_0000131...,1.0,0,0,0,1
4,4,airfield,0,Places365_val_00001650.jpg,Places365_train\airfield\Places365_val_0000165...,0.0,0,0,2,1


In [92]:
data_df_sf = data_df.iloc[pmt_order]
vgg_data_df = vgg_data_df.iloc[pmt_order]
resnet_data_df = resnet_data_df.iloc[pmt_order]
inception_data_df = inception_data_df.iloc[pmt_order]
densenet_data_df = densenet_data_df.iloc[pmt_order]

In [93]:
train_df = data_df_sf[data_df_sf['assign'] == 0]
test_df = data_df_sf[data_df_sf['assign'] == 1]
valid_df = data_df_sf[data_df_sf['assign'] == 2]

vgg_train_df = vgg_data_df[vgg_data_df['assign'] == 0]
vgg_test_df = vgg_data_df[vgg_data_df['assign'] == 1]
vgg_valid_df = vgg_data_df[vgg_data_df['assign'] == 2]

#vgg_train_df = vgg_train_df.sample(frac=1).reset_index(drop=True)
#vgg_test_df = vgg_test_df.sample(frac=1).reset_index(drop=True)
#vgg_valid_df = vgg_valid_df.sample(frac=1).reset_index(drop=True)

resnet_train_df = resnet_data_df[resnet_data_df['assign'] == 0]
resnet_test_df = resnet_data_df[resnet_data_df['assign'] == 1]
resnet_valid_df = resnet_data_df[resnet_data_df['assign'] == 2]

#resnet_train_df = resnet_train_df.sample(frac=1).reset_index(drop=True)
#resnet_test_df = resnet_test_df.sample(frac=1).reset_index(drop=True)
#resnet_valid_df = resnet_valid_df.sample(frac=1).reset_index(drop=True)

inception_train_df = inception_data_df[inception_data_df['assign'] == 0]
inception_test_df = inception_data_df[inception_data_df['assign'] == 1]
inception_valid_df = inception_data_df[inception_data_df['assign'] == 2]

#inception_train_df = inception_train_df.sample(frac=1).reset_index(drop=True)
#inception_test_df = inception_test_df.sample(frac=1).reset_index(drop=True)
#inception_valid_df = inception_valid_df.sample(frac=1).reset_index(drop=True)

densenet_train_df = densenet_data_df[densenet_data_df['assign'] == 0]
densenet_test_df = densenet_data_df[densenet_data_df['assign'] == 1]
densenet_valid_df = densenet_data_df[densenet_data_df['assign'] == 2]

#densenet_train_df = densenet_train_df.sample(frac=1).reset_index(drop=True)
#densenet_test_df = densenet_test_df.sample(frac=1).reset_index(drop=True)
#densenet_valid_df = densenet_valid_df.sample(frac=1).reset_index(drop=True)

In [94]:
vgg_train_features = list(vgg_train_df['feature'])
vgg_train_labels = list(vgg_train_df['label'])

vgg_train_features = np.reshape(vgg_train_features, (len(vgg_train_df), vgg_input_dimension))
vgg_train_labels = np.reshape(vgg_train_labels, (len(vgg_train_df), len(class_names)))

vgg_test_features = list(vgg_test_df['feature'])
vgg_test_labels = list(vgg_test_df['label'])

vgg_test_features = np.reshape(vgg_test_features, (len(vgg_test_df), vgg_input_dimension))
vgg_test_labels = np.reshape(vgg_test_labels, (len(vgg_test_df), len(class_names)))

vgg_valid_features = list(vgg_valid_df['feature'])
vgg_valid_labels = list(vgg_valid_df['label'])

vgg_valid_features = np.reshape(vgg_valid_features, (len(vgg_valid_df), vgg_input_dimension))
vgg_valid_labels = np.reshape(vgg_valid_labels, (len(vgg_valid_df), len(class_names)))

In [95]:
resnet_train_features = list(resnet_train_df['feature'])
resnet_train_labels = list(resnet_train_df['label'])

resnet_train_features = np.reshape(resnet_train_features, (len(resnet_train_df), resnet_input_dimension))
resnet_train_labels = np.reshape(resnet_train_labels, (len(resnet_train_df), len(class_names)))

resnet_test_features = list(resnet_test_df['feature'])
resnet_test_labels = list(resnet_test_df['label'])

resnet_test_features = np.reshape(resnet_test_features, (len(resnet_test_df), resnet_input_dimension))
resnet_test_labels = np.reshape(resnet_test_labels, (len(resnet_test_df), len(class_names)))

resnet_valid_features = list(resnet_valid_df['feature'])
resnet_valid_labels = list(resnet_valid_df['label'])

resnet_valid_features = np.reshape(resnet_valid_features, (len(resnet_valid_df), resnet_input_dimension))
resnet_valid_labels = np.reshape(resnet_valid_labels, (len(resnet_valid_df), len(class_names)))

In [96]:
inception_train_features = list(inception_train_df['feature'])
inception_train_labels = list(inception_train_df['label'])

inception_train_features = np.reshape(inception_train_features, (len(inception_train_df), inception_input_dimension))
inception_train_labels = np.reshape(inception_train_labels, (len(inception_train_df), len(class_names)))

inception_test_features = list(inception_test_df['feature'])
inception_test_labels = list(inception_test_df['label'])

inception_test_features = np.reshape(inception_test_features, (len(inception_test_df), inception_input_dimension))
inception_test_labels = np.reshape(inception_test_labels, (len(inception_test_df), len(class_names)))

inception_valid_features = list(inception_valid_df['feature'])
inception_valid_labels = list(inception_valid_df['label'])

inception_valid_features = np.reshape(inception_valid_features, (len(inception_valid_df), inception_input_dimension))
inception_valid_labels = np.reshape(inception_valid_labels, (len(inception_valid_df), len(class_names)))

In [97]:
densenet_train_features = list(densenet_train_df['feature'])
densenet_train_labels = list(densenet_train_df['label'])

densenet_train_features = np.reshape(densenet_train_features, (len(densenet_train_df), densenet_input_dimension))
densenet_train_labels = np.reshape(densenet_train_labels, (len(densenet_train_df), len(class_names)))

densenet_test_features = list(densenet_test_df['feature'])
densenet_test_labels = list(densenet_test_df['label'])

densenet_test_features = np.reshape(densenet_test_features, (len(densenet_test_df), densenet_input_dimension))
densenet_test_labels = np.reshape(densenet_test_labels, (len(densenet_test_df), len(class_names)))

densenet_valid_features = list(densenet_valid_df['feature'])
densenet_valid_labels = list(densenet_valid_df['label'])

densenet_valid_features = np.reshape(densenet_valid_features, (len(densenet_valid_df), densenet_input_dimension))
densenet_valid_labels = np.reshape(densenet_valid_labels, (len(densenet_valid_df), len(class_names)))

In [98]:
vgg_train_history = vgg_model.fit(vgg_train_features, vgg_train_labels, epochs = 10, batch_size = batch_size, validation_data = (vgg_valid_features, vgg_valid_labels))

Epoch 1/10
21/21 [==============================] - 3s 91ms/step - loss: 3.4596 - accuracy: 0.1682 - val_loss: 1.6554 - val_accuracy: 0.4733
Epoch 2/10
21/21 [==============================] - 1s 62ms/step - loss: 1.0534 - accuracy: 0.6664 - val_loss: 1.2038 - val_accuracy: 0.5933
Epoch 3/10
21/21 [==============================] - 1s 64ms/step - loss: 0.5255 - accuracy: 0.8534 - val_loss: 1.1638 - val_accuracy: 0.6400
Epoch 4/10
21/21 [==============================] - 1s 62ms/step - loss: 0.2377 - accuracy: 0.9470 - val_loss: 1.1080 - val_accuracy: 0.6800
Epoch 5/10
21/21 [==============================] - 1s 60ms/step - loss: 0.0755 - accuracy: 0.9949 - val_loss: 1.0459 - val_accuracy: 0.6800
Epoch 6/10
21/21 [==============================] - 1s 60ms/step - loss: 0.0437 - accuracy: 0.9966 - val_loss: 1.0194 - val_accuracy: 0.6933
Epoch 7/10
21/21 [==============================] - 1s 61ms/step - loss: 0.0212 - accuracy: 0.9976 - val_loss: 1.0611 - val_accuracy: 0.6733
Epoch 8/10
21

In [99]:
resnet_train_history = resnet_model.fit(resnet_train_features, resnet_train_labels, epochs = 10, batch_size = batch_size, validation_data = (resnet_valid_features, resnet_valid_labels))

Epoch 1/10
21/21 [==============================] - 7s 302ms/step - loss: 12.1042 - accuracy: 0.3271 - val_loss: 3.0588 - val_accuracy: 0.6267
Epoch 2/10
21/21 [==============================] - 6s 282ms/step - loss: 0.5313 - accuracy: 0.9022 - val_loss: 2.4196 - val_accuracy: 0.6733
Epoch 3/10
21/21 [==============================] - 6s 274ms/step - loss: 0.1026 - accuracy: 0.9660 - val_loss: 2.3893 - val_accuracy: 0.7400
Epoch 4/10
21/21 [==============================] - 6s 273ms/step - loss: 0.0369 - accuracy: 0.9920 - val_loss: 2.5205 - val_accuracy: 0.7533
Epoch 5/10
21/21 [==============================] - 6s 276ms/step - loss: 0.0274 - accuracy: 0.9938 - val_loss: 2.0130 - val_accuracy: 0.7400
Epoch 6/10
21/21 [==============================] - 6s 272ms/step - loss: 0.0013 - accuracy: 0.9985 - val_loss: 2.0761 - val_accuracy: 0.7533
Epoch 7/10
21/21 [==============================] - 6s 272ms/step - loss: 1.4692e-04 - accuracy: 1.0000 - val_loss: 2.0968 - val_accuracy: 0.7600
E

In [100]:
inception_train_history = inception_model.fit(inception_train_features, inception_train_labels, epochs = 10, batch_size = batch_size, validation_data = (inception_valid_features, inception_valid_labels))

Epoch 1/10
21/21 [==============================] - 4s 125ms/step - loss: 20.8936 - accuracy: 0.1788 - val_loss: 3.2893 - val_accuracy: 0.4933
Epoch 2/10
21/21 [==============================] - 2s 97ms/step - loss: 1.8078 - accuracy: 0.6840 - val_loss: 1.3532 - val_accuracy: 0.7067
Epoch 3/10
21/21 [==============================] - 2s 98ms/step - loss: 0.3926 - accuracy: 0.8937 - val_loss: 1.0649 - val_accuracy: 0.7200
Epoch 4/10
21/21 [==============================] - 2s 95ms/step - loss: 0.0942 - accuracy: 0.9776 - val_loss: 0.9560 - val_accuracy: 0.7200
Epoch 5/10
21/21 [==============================] - 2s 96ms/step - loss: 0.0303 - accuracy: 0.9934 - val_loss: 0.9791 - val_accuracy: 0.7400
Epoch 6/10
21/21 [==============================] - 2s 100ms/step - loss: 0.0134 - accuracy: 0.9980 - val_loss: 0.8675 - val_accuracy: 0.7867
Epoch 7/10
21/21 [==============================] - 2s 101ms/step - loss: 0.0057 - accuracy: 1.0000 - val_loss: 0.9129 - val_accuracy: 0.7800
Epoch 8/1

In [101]:
densenet_train_history = densenet_model.fit(densenet_train_features, densenet_train_labels, epochs = 10, batch_size = batch_size, validation_data = (densenet_valid_features, densenet_valid_labels))

Epoch 1/10
21/21 [==============================] - 4s 127ms/step - loss: 15.0296 - accuracy: 0.2234 - val_loss: 3.5094 - val_accuracy: 0.4933
Epoch 2/10
21/21 [==============================] - 2s 110ms/step - loss: 1.3180 - accuracy: 0.7058 - val_loss: 1.1355 - val_accuracy: 0.7267
Epoch 3/10
21/21 [==============================] - 2s 108ms/step - loss: 0.2032 - accuracy: 0.9419 - val_loss: 1.1364 - val_accuracy: 0.7267
Epoch 4/10
21/21 [==============================] - 2s 108ms/step - loss: 0.0658 - accuracy: 0.9775 - val_loss: 0.8489 - val_accuracy: 0.7333
Epoch 5/10
21/21 [==============================] - 2s 109ms/step - loss: 0.0233 - accuracy: 0.9947 - val_loss: 0.9829 - val_accuracy: 0.7333
Epoch 6/10
21/21 [==============================] - 2s 110ms/step - loss: 0.0059 - accuracy: 1.0000 - val_loss: 0.8547 - val_accuracy: 0.7733
Epoch 7/10
21/21 [==============================] - 2s 108ms/step - loss: 0.0025 - accuracy: 1.0000 - val_loss: 0.8772 - val_accuracy: 0.7667
Epoch

In [102]:
vgg_loss, vgg_acc = vgg_model.evaluate(vgg_test_features, vgg_test_labels)
resnet_loss, resnet_acc = resnet_model.evaluate(resnet_test_features, resnet_test_labels)
inception_loss, inception_acc = inception_model.evaluate(inception_test_features, inception_test_labels)
densenet_loss, densenet_acc = densenet_model.evaluate(densenet_test_features, densenet_test_labels)

10/10 [==============================] - 1s 28ms/step - loss: 0.7688 - accuracy: 0.8233


In [103]:
vgg_prediction_score = vgg_model.predict(vgg_features)
resnet_prediction_score = resnet_model.predict(resnet_features)
inception_prediction_score = inception_model.predict(inception_features)
densenet_prediction_score = densenet_model.predict(densenet_features)

vgg_test_prediction_score = vgg_model.predict(vgg_test_features)
resnet_test_prediction_score = resnet_model.predict(resnet_test_features)
inception_test_prediction_score = inception_model.predict(inception_test_features)
densenet_test_prediction_score = densenet_model.predict(densenet_test_features)

In [104]:
vgg_predicted_label = np.argmax(vgg_prediction_score, axis= -1)
resnet_predicted_label = np.argmax(resnet_prediction_score, axis= -1)
inception_predicted_label = np.argmax(inception_prediction_score, axis= -1)
densenet_predicted_label = np.argmax(densenet_prediction_score, axis= -1)

vgg_test_predicted_label = np.argmax(vgg_test_prediction_score, axis= -1)
resnet_test_predicted_label = np.argmax(resnet_test_prediction_score, axis= -1)
inception_test_predicted_label = np.argmax(inception_test_prediction_score, axis= -1)
densenet_test_predicted_label = np.argmax(densenet_test_prediction_score, axis= -1)

In [105]:
data_df['vgg_predicted_score'] = list(vgg_prediction_score)
data_df['vgg_predicted_label'] = list(vgg_predicted_label)
data_df['resnet_predicted_score'] = list(resnet_prediction_score)
data_df['resnet_predicted_label'] = list(resnet_predicted_label)
data_df['inception_predicted_score'] = list(inception_prediction_score)
data_df['inception_predicted_label'] = list(inception_predicted_label)
data_df['densenet_predicted_score'] = list(densenet_prediction_score)
data_df['densenet_predicted_label'] = list(densenet_predicted_label)


In [106]:
test_df['vgg_predicted_score'] = list(vgg_test_prediction_score)
test_df['vgg_predicted_label'] = list(vgg_test_predicted_label)
test_df['resnet_predicted_score'] = list(resnet_test_prediction_score)
test_df['resnet_predicted_label'] = list(resnet_test_predicted_label)
test_df['inception_predicted_score'] = list(inception_test_prediction_score)
test_df['inception_predicted_label'] = list(inception_test_predicted_label)
test_df['densenet_predicted_score'] = list(densenet_test_prediction_score)
test_df['densenet_predicted_label'] = list(densenet_test_predicted_label)

C:\Users\dayeo\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\dayeo\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\dayeo\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a

In [107]:
data_df.head()

,index,label,label_int,file_names,file_path,assign,vgg_clusters,resnet_clusters,inception_clusters,densenet_clusters,vgg_predicted_score,vgg_predicted_label,resnet_predicted_score,resnet_predicted_label,inception_predicted_score,inception_predicted_label,densenet_predicted_score,densenet_predicted_label
0,0,airfield,0,Places365_val_00000435.jpg,Places365_train\airfield\Places365_val_0000043...,1.0,0,0,2,1,"[0.99974674, 2.024817e-07, 2.0296426e-07, 1.22...",0,"[1.0, 5.1224816e-25, 1.07096845e-19, 2.079632e...",0,"[1.0, 7.154506e-16, 7.5793236e-17, 7.346465e-1...",0,"[0.9999994, 9.487863e-10, 4.1840316e-12, 1.336...",0
1,1,airfield,0,Places365_val_00001163.jpg,Places365_train\airfield\Places365_val_0000116...,0.0,1,0,2,1,"[0.9992986, 8.948807e-07, 9.010984e-08, 6.5604...",0,"[0.99999917, 8.4249674e-27, 3.1796505e-16, 1.3...",0,"[1.0, 2.5554897e-15, 2.025256e-15, 1.160988e-1...",0,"[1.0, 1.2707332e-10, 1.8986851e-10, 1.476803e-...",0
2,2,airfield,0,Places365_val_00001257.jpg,Places365_train\airfield\Places365_val_0000125...,0.0,0,0,2,1,"[0.9999987, 2.3733987e-08, 2.0950275e-09, 1.19...",0,"[1.0, 1.1581774e-21, 1.379611e-20, 4.4922042e-...",0,"[1.0, 1.737587e-16, 9.750289e-17, 7.385888e-18...",0,"[0.9999999, 3.6842807e-11, 1.7205354e-10, 1.20...",0
3,3,airfield,0,Places365_val_00001316.jpg,Places365_train\airfield\Places365_val_0000131...,1.0,0,0,0,1,"[0.999113, 2.8065997e-05, 0.00021257324, 2.998...",0,"[1.0, 9.578816e-11, 1.1418956e-09, 7.918696e-2...",0,"[1.0, 2.3255526e-08, 4.8776386e-08, 3.4163753e...",0,"[0.99998033, 2.7402982e-07, 9.795284e-07, 1.41...",0
4,4,airfield,0,Places365_val_00001650.jpg,Places365_train\airfield\Places365_val_0000165...,0.0,0,0,2,1,"[0.99860483, 5.6103386e-06, 9.597087e-07, 1.38...",0,"[1.0, 8.133172e-15, 6.897089e-14, 6.3469583e-3...",0,"[0.9965334, 0.003460946, 5.2859906e-07, 2.8672...",0,"[0.99999726, 1.6226517e-07, 1.4607366e-09, 1.0...",0


In [108]:
test_df.head()

,index,label,label_int,file_names,file_path,assign,vgg_clusters,resnet_clusters,inception_clusters,densenet_clusters,vgg_predicted_score,vgg_predicted_label,resnet_predicted_score,resnet_predicted_label,inception_predicted_score,inception_predicted_label,densenet_predicted_score,densenet_predicted_label
497,497,bus-station-indoor,4,Places365_val_00035800.jpg,Places365_train\bus-station-indoor\Places365_v...,1.0,2,1,0,2,"[4.2226646e-05, 0.0056469585, 0.00025247058, 0...",5,"[9.855952e-11, 2.3157393e-10, 9.423802e-09, 1....",4,"[7.4550755e-08, 0.014703062, 0.0027483976, 0.0...",9,"[5.928262e-07, 2.1464275e-05, 1.067274e-05, 1....",9
71,71,airfield,0,Places365_val_00027647.jpg,Places365_train\airfield\Places365_val_0002764...,1.0,1,0,2,1,"[0.99936205, 1.1436932e-06, 1.3846201e-06, 7.9...",0,"[1.0, 5.9804993e-18, 1.3663284e-13, 5.103214e-...",0,"[1.0, 2.5019642e-15, 5.8192743e-18, 3.3786032e...",0,"[1.0, 1.753549e-11, 1.053702e-10, 2.6927574e-1...",0
287,287,aquarium,2,Places365_val_00031095.jpg,Places365_train\aquarium\Places365_val_0003109...,1.0,0,0,2,2,"[0.00016762875, 0.00032846796, 0.9970776, 0.00...",2,"[1.8263136e-13, 2.934561e-13, 0.99999976, 5.85...",2,"[3.355493e-06, 0.0060943607, 0.94251704, 1.259...",2,"[1.0641062e-05, 0.0005633362, 0.99920315, 1.56...",2
100,100,amusement-park,1,Places365_val_00000381.jpg,Places365_train\amusement-park\Places365_val_0...,1.0,0,0,0,1,"[0.0002024736, 0.9775283, 0.0009571813, 1.9317...",1,"[1.9207837e-14, 1.0, 1.178999e-14, 2.2829999e-...",1,"[4.0718337e-06, 0.9991819, 1.2813993e-05, 2.27...",1,"[2.8215196e-05, 0.9970108, 0.00014820223, 1.65...",1
679,679,church-outdoor,6,Places365_val_00028877.jpg,Places365_train\church-outdoor\Places365_val_0...,1.0,1,0,2,2,"[9.599688e-06, 0.0233073, 7.613521e-06, 3.1691...",6,"[8.38315e-24, 5.8211065e-14, 4.427811e-19, 6.5...",6,"[4.6280235e-10, 3.6205797e-08, 7.296379e-13, 1...",6,"[2.431938e-08, 1.935451e-05, 3.2308034e-08, 2....",6


In [109]:
len(test_df[test_df['label_int'] == 3])

20

In [110]:
result = data_df.to_json(orient="records")
parsed = json.loads(result)
with open("places_trained_data_result2.json", "w") as outfile:
  json.dump(parsed, outfile, indent = 4)

In [111]:
test_result = test_df.to_json(orient="records")
parsed = json.loads(test_result)
with open("places_trained_test_result2.json", "w") as outfile:
  json.dump(parsed, outfile, indent = 4)

Saliency Map

In [112]:
def saliency_map(model, imgs, last_conv_name):
  with tf.GradientTape() as tape:
    last_conv_layer = model.get_layer(last_conv_name)
    print(last_conv_layer)
    iterate = tf.keras.models.Model([model.inputs], [model.output, last_conv_layer.output])
    print(model.inputs)
    print(model.output)
    print(last_conv_layer.output)
    model_out, last_conv_layer = iterate(imgs)
    class_out = model_out[:, np.argmax(model_out[0])]
    grads = tape.gradient(class_out, last_conv_layer)
    pooled_grads = tf.keras.backend.mean(grads, axis = (0,1,2))

  heatmap = tf.reduce_mean(tf.multiply(pooled_grads, last_conv_layer), axis = -1)

  heatmap = np.maximum(heatmap, 0)
  heatmap /= np.max(heatmap)
  heatmap = heatmap.reshape((7,7))

  return heatmap

In [113]:
vgg_full_model = vgg_layer
flatten = keras.layers.Flatten()(vgg_full_model.layers[-1].output)
dense1 = keras.layers.Dense(512, activation='relu')(flatten)
dense2 = keras.layers.Dense(512, activation = 'relu')(dense1)
output = keras.layers.Dense(20, activation = 'softmax')(dense2)
vgg_full_model = keras.models.Model(inputs = vgg_full_model.inputs, outputs = output)

In [114]:
resnet_full_model = resnet_layer
flatten = keras.layers.Flatten()(resnet_full_model.layers[-1].output)
dense1 = keras.layers.Dense(512, activation='relu')(flatten)
dense2 = keras.layers.Dense(512, activation = 'relu')(dense1)
output = keras.layers.Dense(20, activation = 'softmax')(dense2)
resnet_full_model = keras.models.Model(inputs = resnet_full_model.inputs, outputs = output)

In [115]:
inception_full_model = inception_layer
flatten = keras.layers.Flatten()(inception_full_model.layers[-1].output)
dense1 = keras.layers.Dense(512, activation='relu')(flatten)
dense2 = keras.layers.Dense(512, activation = 'relu')(dense1)
output = keras.layers.Dense(20, activation = 'softmax')(dense2)
inception_full_model = keras.models.Model(inputs = inception_full_model.inputs, outputs = output)

In [116]:
inception_full_model.compile(optimizer=tf.keras.optimizers.Adam(), loss = tf.keras.losses.CategoricalCrossentropy(), metrics=['accuracy'])

In [117]:
inception_full_model.compiled_loss._losses

In [118]:
inception_full_model.save("inception_full_model.h5")

In [119]:
densenet_full_model = densenet_layer
flatten = keras.layers.Flatten()(densenet_full_model.layers[-1].output)
dense1 = keras.layers.Dense(512, activation='relu')(flatten)
dense2 = keras.layers.Dense(512, activation = 'relu')(dense1)
output = keras.layers.Dense(20, activation = 'softmax')(dense2)
densenet_full_model = keras.models.Model(inputs = densenet_full_model.inputs, outputs = output)

In [120]:
inception_processed_imgs[id].shape

NameError: name 'inception_processed_imgs' is not defined

In [ ]:
inception_layer.layers[0].output

<KerasTensor: shape=(None, 224, 224, 3) dtype=float32 (created by layer 'input_5')>

In [ ]:
model_out, last_conv_layer = iterate(imgs)

In [ ]:
id = 155

cur_img = np.expand_dims(inception_processed_imgs[id], axis = 0)
hmap = saliency_map(inception_full_model, cur_img, inception_conv_layers[-1][0])
plt.matshow(hmap)

img = cv2.imread(filepath[id])

INTENSITY = 0.5

heatmap = cv2.resize(hmap, (img.shape[1], img.shape[0]))
heatmap = cv2.applyColorMap(np.uint8(255*heatmap), cv2.COLORMAP_JET)

salient_img = heatmap * INTENSITY + img
cv2.imwrite('dataset/salient_img.jpg', salient_img)

[<KerasTensor: shape=(None, 224, 224, 3) dtype=float32 (created by layer 'input_5')>]
KerasTensor(type_spec=TensorSpec(shape=(None, 20), dtype=tf.float32, name=None), name='dense_23/Softmax:0', description="created by layer 'dense_23'")
KerasTensor(type_spec=TensorSpec(shape=(None, 5, 5, 1536), dtype=tf.float32, name=None), name='conv_7b/Conv2D:0', description="created by layer 'conv_7b'")


ValueError: Input 0 is incompatible with layer model_6: expected shape=(None, 224, 224, 3), found shape=(1, 299, 299, 3)

In [ ]:
kkk = np.expand_dims(resnet_processed_imgs[234], axis = 0)
kkk.shape

h = saliency_map(resnet_full_model, kkk, resnet_conv_layers[-1][0])

plt.matshow(h)
plt.show()

iii = cv2.imread(filepaths[234])

INTENSITY = 0.5
heatmap = cv2.resize(h, (iii.shape[1], iii.shape[0]))
heatmap = cv2.applyColorMap(np.uint8(255*heatmap), cv2.COLORMAP_JET)

iii = heatmap * INTENSITY + iii

cv2.imwrite('dataset/cam1.jpg', iii)
